# exercise with pandas

every "..." should be replaced with a line of code

In [ ]:
import pandas as pd
import os

# Use PUIdata environmental variable to store data
PUIdata = os.getenv('PUIDATA')

print (PUIdata)

Read what the PLUTO data is [here](https://www.wired.com/2013/08/nyc-pluto-data-map-party/)

Find the PLUTO dataset online, find the cvs file data (not the shape file), it should be about 46M

Download it as a zip file, unpack it in PUIDATAEach borough will be in a different file. The goal is to find land plot categories of which there is a higher number in Manhattan (MN) than in Brooklyn (BK)

Read in csv data as pandas dataframes (with pd.read_csv() )

In a Pandas DataFrame each column is a _series_ (a pandas datatype) to manipulate the data here you will mostly use methods (functions that are defined for a specific datatype and invoked with the _object.method()_ syntax) of the DataFrames (e.g. df.groupby() ) or of the series therein (e.g. series.astype() )

In [ ]:
!curl -O https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_18v1.zip
!unzip nyc_pluto_18v1.zip -d $PUIdata

In [ ]:
!ls $PUIdata

# read in te Manhattan and Brooklyn data in 2 separate dataframes w pandas

## show the first 10 lines of each df

In [ ]:
# show manhattan 10 rows
mn = pd.read_csv(PUIdata + '/PLUTO_for_WEB/MN_18v1.csv')
mn.head(10)

In [ ]:
# show brooklyn 10 rows
bk = pd.read_csv(PUIdata + '/PLUTO_for_WEB/BK_18v1.csv')
bk.head(10)

## display all columns in each file

In [ ]:
# show bk
bk.columns.get_values()

In [ ]:
# show mn
mn.columns.get_values()

# drop all columns but ZipCode and LandUse

In [ ]:
# drop columns here. Use the .drop method of pandas dataframes, i.e. something like bk.drop()... 
# make sure you drop "in place" and that you drop along the right axis - i.e. the columns. 
# you can use bk.drop? to the help with this function 
mn_del_cols = []
mn_cols = mn.columns.get_values()
for col in mn_cols:
    if col != 'ZipCode' and col != 'LandUse':
        mn_del_cols.append(col)
mn.drop(mn_del_cols, axis=1, inplace=True)
mn.head()

In [ ]:
# Drop columns from bk df
bk_del_cols = []
bk_cols = bk.columns.get_values()
for col in bk_cols:
    if col != 'ZipCode' and col != 'LandUse':
        bk_del_cols.append(col)
bk.drop(bk_del_cols, axis=1, inplace=True)
bk.head()

# use groupby to count the building per LandUse code 

In [ ]:
mngrouped = mn.groupby('LandUse').count().reset_index()
mngrouped = mngrouped.rename(columns = {'ZipCode':'Count'})
mngrouped.head(100)

In [ ]:
# group Manhattan by land use. Make sure you understand how to use the groupby() method:
# this method returns a group object. those objects have methods and some of these methods return dataframes. 
# What do you want to do? you want to count all objects per each land use type. 
# So you want to use .groupby() to group by LandUse and then .count() to get the number of element per LandUse
# Other methods include .mean() to get the mean of a group, .std(), 
# and a generic method .apply() to which you can pass any function that operates on a column.
# All these methods return dataframes 
# (hint: use also reset_index) for each df
mngrouped = mn.groupby('LandUse').count().reset_index()
mngrouped.rename(columns = {'ZipCode':'CountMN'}, inplace=True)
mngrouped

In [ ]:
# group BK by land use
bkgrouped = bk.groupby('LandUse').count().reset_index()
bkgrouped.rename(columns = {'ZipCode':'CountBK'}, inplace=True)
bkgrouped

# convert the landuse to integer 

In [ ]:
# convert LandUse to int using the .astype() method of the series
mngrouped['LandUse'] = mngrouped['LandUse'].astype(int)
bkgrouped['LandUse'] = bkgrouped['LandUse'].astype(int)
bkgrouped.head()

# merge the dataframes on land use 

In [ ]:
# merge the dataframes on LandUse. You can use the pandas function merge ( _pd.merge)
both = bkgrouped.merge(mngrouped, how='outer', on='LandUse')
both

# rename things so that you have descriptive names

## rename the columns with descriptive names. 


In [ ]:
# Use the df.rename() method (hint: it takes a dictionary)
# now you can just call "both" instead of "both.head()" since the file should only have 11 columns
both.rename(columns = {'CountBK':'landuse_count_bk', 'CountMN':'landuse_count_mn'}, inplace=True)
both

## find online the LandUse data dictionary and replace code w the verbose description of it


In [ ]:
# set up Land Use values to descriptive tag
# Found descriptions here: 
# https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/pluto_datadictionary.pdf?r=18v1
both.loc[0,'LandUse'] = 'One & Two Family Buildings'
both.loc[1,'LandUse'] = 'Multi-Family Walk-Up Buildings'
both.loc[2,'LandUse'] = 'Multi-Family Elevator Buildings'
both.loc[3,'LandUse'] = 'Mixed Residential & Commercial Buildings'
both.loc[4,'LandUse'] = 'Commercial & Office Buildings'
both.loc[5,'LandUse'] = 'Industrial & Manufacturing Buildings'
both.loc[6,'LandUse'] = 'Transportation & Utility'
both.loc[7,'LandUse'] = 'Public Facilities & Institutions'
both.loc[8,'LandUse'] = 'Open Space & Outdoor Recreation'
both.loc[9,'LandUse'] = 'Parking Facilities'
both.loc[10,'LandUse'] = 'Vacant Land'

both

# create a feature (a column, a series) with the ratio of number of buildings in BK vs MN and find out in which LandUse class Manhattan has more buildings than BK 

In [ ]:
# calculate the ratio of BK to Man buildings per LandUse category
both['luratio'] = both.landuse_count_mn / both.landuse_count_bk
both

In [ ]:
# defined function that returns True if the ratio is larger than 0, False otherwise. 
# Use the series.apply() method to apply that function to the luratio series you jusrt created
def ratio_check(ratio):
    if ratio > 1:
        return True
    else:
        return False

both['Manhattan wins'] = both['luratio'].apply(ratio_check) 
both

# comment on what you found out. Does it make sense? 

Since we are comparing absolute values between Brooklyn and Manhattan, it would make sense that Brooklyn will have more instances of many land types since Brooklyn's population is greater than Manhattan's by approximately one million people. The one land use category for which Manhattan has more properties (multi-family elevator buildings) makes intuitive sense because Manhattan has more high-rises due to the limited land area available for property development on the island.